   ![](files/images/cover_f2py.png)  

## f2py
* f2py allows to build an extension module that interfaces Python to Fortran 77/90/95 routines
* Let's write a Fortran function to interface a simple array_multiply routine


## matmul example

In [ ]:
%%writefile mult.f90

subroutine fmult(a,b,c,n)
implicit none
real*8 :: a(*)
real*8 :: b(*)
real*8 :: c(*)
integer :: n, i
do i =1,n
  c(i) = a(i) * b(i)
enddo
end

In [ ]:
!f2py3 -c -m mult mult.f90

In [ ]:
# now we can import the module mult.so
import mult
print(mult.fmult.__doc__)

In [ ]:
a = np.ones(10000)+ 3 
b = np.ones(10000)+ 1.5
c = np.empty(a.size)
mult.fmult(a,b,c,a.size)
c

## f2py: improving interface
* One can improve the interface automatically built to make it more pythonic

In [ ]:
!f2py -h mult.pyf -m mult mult.f90

In [ ]:
!cat mult.pyf

## Let's adapt the interface to be more pythonic

In [ ]:
%%writefile mult.pyf

!    -*- f90 -*-
! Note: the context of this file is case sensitive.

python module mult ! in 
    interface  ! in :mult
        subroutine fmult(a,b,c,n) ! in :mult:mult.f90
            real*8 dimension(n) :: a
            real*8 dimension(n) :: b
            real*8 intent(out), dimension(n) :: c
            integer intent(hide), depend(a) :: n=len(a)
        end subroutine fmult
    end interface 
end python module mult

! This file was auto-generated with f2py (version:2).
! See http://cens.ioc.ee/projects/f2py2e/

In [ ]:
!rm mult.cpython-34m.so
!f2py3 -c -m mult mult.pyf mult.f90

In [ ]:
import mult
print(mult.fmult.__doc__)

## f2py: improving the interface 

* Inserting directives in fortran source to make the function more pythonic

In [ ]:
%%writefile mult2.f90

subroutine fmult(a,b,c,n)
implicit none
real*8 :: a(n)
real*8 :: b(n)
real*8 :: c(n)
integer :: n, i
!f2py intent(hide), depend(a) :: n=len(a)
!f2py real*8 :: a(n)
!f2py real*8 :: b(n)
!f2py real*8, intent(out) :: c(n)
do i =1,n
  c(i) = a(i) * b(i)
enddo
end


In [ ]:
!diff mult.f90 mult2.f90

In [ ]:
!f2py3 -c -m mult2 mult2.f90

In [ ]:
import mult2
print(mult2.fmult.__doc__)

In [ ]:
import numpy as np

In [ ]:
a = np.array([1, 3, 4])
b = np.array([2, 5, 1.5])
c = mult2.fmult(a, b) 
c

In [64]:
a = np.arange(1, 10**6, 1.1)
b = a.copy()

In [65]:
%timeit a*b

100 loops, best of 3: 3.28 ms per loop


In [66]:
%timeit mult2.fmult(a,b)

100 loops, best of 3: 3.9 ms per loop


## second example: mandelbrot

This fractal is defined by the iteration

$$z ← z^2 + c$$

where z and c are complex variables. This expression is iterated; if z stays finite, c belongs to the Mandelbrot set


In [67]:
%%writefile mandel.f90

subroutine single_point_mandelbrot(z,c,zout)
    ! *************************************************
    ! * compute single point mandlebrot *
    ! *************************************************
    implicit none
    integer :: I 
    complex*16 :: z, c, zout
    do i = 1,100
        z = z*z + c
        if ((real(z)**2 + aimag(z)**2 ) .gt. 1000.0) then
           exit
        endif 
    enddo
    zout = z
end subroutine


Overwriting mandel.f90


## mandelbrot/2
Let us get the new module a more pythonic interface


In [68]:
%%writefile mandel2.f90

subroutine single_point_mandelbrot(z,c,zout)
    ! *************************************************
    ! * compute single point mandlebrot *
    ! *************************************************
    implicit none
    integer :: I 
    complex*16 :: z, c, zout 
    !f2py complex*16, intent(out) :: zout 
    do i = 1,100
        z = z*z + c
        if ((real(z)**2 + aimag(z)**2 ) .gt. 1000.0) then
           exit
        endif 
    enddo
    zout = z
end subroutine

Overwriting mandel2.f90


In [ ]:
!f2py3 -c -m mandel mandel2.f90

In [ ]:
import mandel
print(mandel.single_point_mandelbrot.__doc__)

## mandelbrot/3
* Let's test the module (and the interface)


In [ ]:
z = complex(1,2)
c = complex(1,2)
zout = mandel.single_point_mandelbrot(z,c)
zout


## mandelbrot/4
* Now we can employ our function
* First, we need to wrap our function


In [ ]:
def myfunc(a,b):
    return mandel.single_point_mandelbrot(a,b)

* And then vectorize it


In [ ]:
vfunc = np.vectorize(myfunc) #now we can use madel.single_point_mandelbrot as a ufunc 

* Let's build the input points ...

In [ ]:
x = np.linspace(-1.7, 0.6, 1000)
y = np.linspace(-1.4, 1.4, 1000)
xx, yy = np.meshgrid(x, y)  

* ... and then get the complex numbers


In [69]:
def f(aa, bb):
    return aa + 1j*bb  

ff = f(xx, yy)  # build a 2d array of complex numbers

*  Now we can use our new function and plot the results

In [ ]:
z = vfunc(ff,ff) # just used our single_point_mandelbrot func
%matplotlib inline
import matplotlib.pyplot as plt
plt.imshow(abs(z)**2 < 1000, extent=[-1.7, 0.6, -1.4, 1.4])
plt.gray()
plt.show()